# **DESAFIO 1 G2** 
# **Dataset Properati - Analisis exploratorio y acondicionamiento de datos.**
---

<a id="section_toc"></a> 
## Tabla de Contenidos

[Intro](#section_intro)

Dataset

$\hspace{.5cm}$[Observaciones generales](#section_og)
  
$\hspace{.5cm}$[Procesos generales](#section_pg)

Imputaciones

$\hspace{.5cm}$[Expresiones regulares](#section_re)

$\hspace{.9cm}$[Principios generales](#section_re_pg)

$\hspace{.9cm}$[Precios](#section_regexp_pre)

$\hspace{.9cm}$[Ambientes](#section_regexp_amb)

$\hspace{.9cm}$[Amenities](#section_regexp_ame)


---

<a id="section_intro"></a> 
## Intro

[volver a TOC](#section_toc)

Carga de DataFrame y exploracion inicial

In [1]:
import pandas as pd
import numpy as np
import re

In [3]:
data_location = '../properati.zip'
data = pd.read_csv(data_location, compression='zip', encoding="utf-8")

In [ ]:
pd.set_option('display.max_colwidth', 1500)
pd.set_option('display.max_rows', 200)

<a id="section_og"></a> 
### Observaciones Generales

[volver a TOC](#section_toc)

datosCol(serie) - Funcion que retorna diccionario con los datos de una Serie / Columna del DataFrame a analizar:

Campos:
 - nombre columna.
 - tipo de dato Pandas serie.
 - cantidad de registros en la serie.
 - cantidad de nulos en la serie.
 - cantidad de NO nulos en la serie.
 - porcentaje de No nulos en la serie.
 - cantidad de valores unicos en la serie.
 - valor mas repetido en la serie.
 - valor maximo en la serie.
 - valor minimos en la serie.
 - media, mediana y desvio estandar en la serie.
 - distribucion de cuantiles.
 - tipos de datos sobre los valores incluidos en la serie.

In [4]:
def datosCol(col=None):
    try:
        if col.dtype != object:
            return {
                    'nombre': col.name,
                    'series_dt': col.dtype,
                    'registros': int(len(col)), 
                    'nulos': int(len(col) - col.describe()[0]), 
                    'nulos_porcentaje': round(col.isnull().sum() / col.size*100, 2),
                    'nonulos': int(col.describe()[0]),
                    'nonulos_porcentraje': round(col.describe()[0] / col.shape[0], 2),
                    'unicos': int(len(col.value_counts())),
                    'mas_repetido': col.value_counts().index[0],
                    'minimo': round(col.min(), 2),
                    'maximo': round(col.max(), 2),
                    'media': round(col.describe()[1], 2),
                    'mediana': col.median(),
                    'desvio': round(col.describe()[2], 2),
                    'q25': col.describe()[4],
                    'q50': col.describe()[5],
                    'q75': col.describe()[6],
                    'values_dt': [x for x in col.apply(lambda x: type(x)).value_counts().index]
                   }
        else:
            return {
                    'nombre': col.name,
                    'series_dt': col.dtype,
                    'registros': int(col.describe()[0]),
                    'nulos': col.isnull().sum(),
                    'nulos_porcentaje': round(col.isnull().sum() / col.size*100, 2),
                    'nonulos': int(col.notnull().sum()),
                    'nonulos_porcentraje': round(col.notnull().sum() / col.size*100, 2),
                    'unicos': int(col.describe()[1]), 
                    'mas_repetido': col.describe()[2],
                    'nulos': col.isnull().sum(),
                    'frecuencia': int(col.describe()[3]),
                    'values_dt': [x for x in col.apply(lambda x: type(x)).value_counts().index]
                   }
    except:
        return 'use a valid pandas Series'

In [6]:
data_info = pd.DataFrame([datosCol(data[x]) for x in data.columns ])

In [7]:
data_info

,nombre,series_dt,registros,nulos,nulos_porcentaje,nonulos,nonulos_porcentraje,unicos,mas_repetido,minimo,maximo,media,mediana,desvio,q25,q50,q75,values_dt,frecuencia
0,Unnamed: 0,int64,121220,0,0.00,121220,1.00,121220,2047,0.00,1.212190e+05,60609.50,6.060950e+04,34993.34,3.030475e+04,6.060950e+04,9.091425e+04,[<class 'int'>],NaN
1,operation,object,121220,0,0.00,121220,100.00,1,sell,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[<class 'str'>],121220.0
2,property_type,object,121220,0,0.00,121220,100.00,4,apartment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[<class 'str'>],71065.0
3,place_name,object,121197,23,0.02,121197,99.98,1060,Córdoba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[<class 'str'>, <class 'float'>]",9254.0
4,place_with_parent_names,object,121220,0,0.00,121220,100.00,1164,|Argentina|Santa Fe|Rosario|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[<class 'str'>],8504.0
5,country_name,object,121220,0,0.00,121220,100.00,1,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[<class 'str'>],121220.0
6,state_name,object,121220,0,0.00,121220,100.00,28,Capital Federal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[<class 'str'>],32316.0
7,geonames_id,float64,121220,18717,15.44,102503,0.85,646,3.83857e+06,3427208.00,6.948895e+06,3574442.32,3.433910e+06,354130.62,3.430234e+06,3.433910e+06,3.836668e+06,[<class 'float'>],NaN
8,lat-lon,object,69670,51550,42.53,69670,57.47,47203,"-34.4026444,-58.6684776",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[<class 'str'>, <class 'float'>]",312.0
9,lat,float64,121220,51550,42.53,69670,0.57,46495,-34.4026,-54.82,4.550000e+00,-34.63,-3.459799e+01,1.98,-3.466907e+01,-3.459799e+01,-3.444130e+01,[<class 'float'>],NaN


Exploracion general
<a id="section_pg"></a> 
### Procesos Generales

[volver a TOC](#section_pg)

In [39]:
# renombro la columna que no tiene nombre
data = data.rename({           
    'Unnamed: 0': 'Id_caso'
}, axis=1)

In [40]:
# valores 
medida_property_type = data.property_type.value_counts()
print(medida_property_type) 
# ...........terminar los otros

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64


### Exploracion sobre ubicacion de propiedades
---

**Analisis campos ubicacion:** `place_name` `place_with_parent_names` `lat-lon` `geonames_id`

In [ ]:
Campo `geonames_id`:
 - Comprobacion: validacion de asignacion de mismo geonames_id a distintos valores place_name_new
 - Normalizacion de valores nulos

Campo `lat-lon`:
 
Campo `tipo_de_cambio`:
 
Campo `precio_m2`:

Validacion y comparacion de consistencia entre:
 - `place_with_parent_names`
 - `place_name`

Vamos a determinadar si la variable `place_name` tiene un dato 'confiable' o 'completo', para eso separamos la variable `place_with_parent_names` en secciones, por el simbolo | (pipe) separo la variable place_with_parent_names, sumando campos nuevos y ya trabajando sobre el nuevo dataframe

In [12]:
data_trabajo = data.join(data["place_with_parent_names"].str.split('|', expand=True).rename(columns={
                                                                                                1:'Pais', 
                                                                                                2:'Zona', 
                                                                                                3:'Partido_barrio', 
                                                                                                4:'Localidad', 
                                                                                                5:'Obs_localidad', 
                                                                                                6:'Descarte'}))
data_trabajo.drop(['Descarte'], axis='columns', inplace=True)

Validamos que la variable `place_name` sea igual a el ultimo valor informado sobre `place_with_parent_names` ya separado por el proceso anterior.
Se desde la ultima variable hasta encontrar un NO nulo y se graba el valor:

 1. Obs_localidad
 2. Localidad
 3. Partido_barrio
 4. Zona
 
Creacion de una nueva serie `place_name_new` que se completara a medida que valide que cada variable tenga valor empezando desde la ultima seccion.

In [17]:
data_trabajo["place_name_new"] = 'validar'

#para cada variable nuevo valido null o ''
# creo mascara de los casos donde la varible 'x' tenga null o ''
mask_data_Obs_localidad_null =  (data_trabajo.Obs_localidad.isnull()) | (data_trabajo.Obs_localidad == '')

# creo otra mascara con origen en la mascara enterior para quedarme con los 
# que tienen valor en la variable 'x'
mask_data_Obs_localidad_notnull = mask_data_Obs_localidad_null == False

Actualización de `place_name_new`

1. Obs_localidad

In [25]:
print ('Cantidad de casos con Obs_localidad informada: ' + str(mask_data_Obs_localidad_notnull.sum()))

# creo una mascara sobre la variable que voy a ir completando con el valor obtenido
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original :' + str(mask_data_place_name_new_validar.sum()))

# actualizo los valores sobre mi nueva variable con los que encontré en la variable 'x'
data_trabajo.loc[mask_data_Obs_localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Obs_localidad

# actualizo la mascara sobre mi nueva variable para contar y ver si todo resultó ok!
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))

# la cantidad de casos sobre la variable nueva con valor 'validar' era de 121197
# La cantidad de casos sobre la misma variable con valor 'validar' despues de actualizar fue de 120649
# El delta corresponde a los valores con origen en el primer campo Obs_localidad ... 548

Cantidad de casos con Obs_localidad informada: 548
Cantidad de casos con place_name_new original :0
Cantidad de casos con place_name_new posterior para analizar: 0


2. Localidad

In [30]:
mask_data_Localidad_null =  (data_trabajo.Localidad.isnull()) | (data_trabajo.Localidad == '')
mask_data_Localidad_notnull = mask_data_Localidad_null == False
print ('Cantidad de casos con Localidad informada: ' + str(mask_data_Localidad_notnull.sum()))

mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' +  str(mask_data_place_name_new_validar.sum()))

data_trabajo.loc[mask_data_Localidad_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Localidad
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))

Cantidad de casos con Localidad informada: 40440
Cantidad de casos con place_name_new original: 0
Cantidad de casos con place_name_new posterior para analizar: 0


3. Partido_barrio

In [31]:
mask_data_Partido_barrio_null =  (data_trabajo.Partido_barrio.isnull()) | (data_trabajo.Partido_barrio == '')
mask_data_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
print ('Cantidad de casos con Partido_barrio informada: ' + str(mask_data_Partido_barrio_notnull.sum()))

mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))

data_trabajo.loc[mask_data_Partido_barrio_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Partido_barrio
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))

Cantidad de casos con Partido_barrio informada: 116440
Cantidad de casos con place_name_new original: 0
Cantidad de casos con place_name_new posterior para analizar: 0


4. Zona

In [32]:
mask_data_Zona_null =  (data_trabajo.Zona.isnull()) | (data_trabajo.Zona == '')
mask_data_Zona_notnull = mask_data_Zona_null == False
print ('Cantidad de casos con Zona informada: ' + str(mask_data_Zona_notnull.sum()))

mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new original: ' + str(mask_data_place_name_new_validar.sum()))

data_trabajo.loc[mask_data_Zona_notnull&mask_data_place_name_new_validar, "place_name_new"] = data_trabajo.Zona
mask_data_place_name_new_validar = data_trabajo.place_name_new == 'validar'
print ('Cantidad de casos con place_name_new posterior para analizar: ' + str(mask_data_place_name_new_validar.sum()))

Cantidad de casos con Zona informada: 121220
Cantidad de casos con place_name_new original: 0
Cantidad de casos con place_name_new posterior para analizar: 0


Cantidad casos que coinciden entre place_name_new y place_name

La coincidencia de registros es un indicador de coincidencia

In [34]:
# valido si place_name_new da igual que place_name, si contamos lo mismo es que da igual
mask_place_name_validacion =  data_trabajo.place_name_new == data_trabajo.place_name
print(mask_place_name_validacion.value_counts(), '\n')  

mask_place_name_validacion_falso = mask_place_name_validacion == False
print ('Vemos que casuiticas encontramos sobre los casos que no encontramos:', '\n')
print(data_trabajo.loc[mask_place_name_validacion_falso, ["Id_caso","place_with_parent_names","place_name", "place_name_new"]].\
     groupby(["place_with_parent_names"])["Id_caso"].count())
print('Vemos que son todos del mismo place_with_parent_names de Tigre', '\n')

print ('Los normalizamos sobre la variable place_name_new', '\n')
data_trabajo.loc[mask_place_name_validacion_falso, "place_name_new"] = 'Tigre'
data_trabajo.loc[mask_place_name_validacion_falso, "Localidad"] = 'Tigre'
print ('Validamos la actualización:', '\n')
print(data_trabajo.loc[mask_place_name_validacion_falso, ["Id_caso", "place_name_new"]]\
     .groupby(["place_name_new"])["Id_caso"].count())

True     121197
False        23
dtype: int64 

Vemos que casuiticas encontramos sobre los casos que no encontramos: 

place_with_parent_names
|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||    23
Name: Id_caso, dtype: int64
Vemos que son todos del mismo place_with_parent_names de Tigre 

Los normalizamos sobre la variable place_name_new 

Validamos la actualización: 

place_name_new
Tigre    23
Name: Id_caso, dtype: int64


---

`calidad_dato` nueva Serie para validar la calidad del dato `place_with_parent_names` luego de separarlo 

Dependiendo de la zona que estamos analizando entendemos que la cantidad de info que necesitamos para:

 - `['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']` el mínimo tendria que ser _Localidad_.
 - `['Buenos Aires Costa Atlántica']` y en especial MDQ el mínimo tendria que ser _Localidad_.
 - el resto de las zonas _Partido_barrio_.

Se actualizará la Serie creada (`calidad_dato`) a True que identificará los casos sobre los que necesitamos mayor información:

Para `['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']` se creara una lista de valores.

In [35]:
data_trabajo["calidad_dato"] = False

# zonas a traves de una lista de valores
valores_zona = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur']

# mascara sobre la lista de valores y que localidad este nulo o ''
mask_Localidad = ([x in valores_zona for x in data_trabajo.Zona]) & ((data_trabajo.Localidad.isnull())|(data_trabajo.Localidad == ''))

# actualizo la variable creada sobre los True por la mascara
data_trabajo.loc[mask_Localidad, "calidad_dato"] = True

# caso especial de MDQ
valores_zona_mdq = ['Buenos Aires Costa Atlántica']
mask_Localidad_mdq = ([x in valores_zona_mdq for x in data_trabajo.Zona])\
& (data_trabajo.Partido_barrio == 'Mar del Plata') & ((data_trabajo.Localidad.isnull()) | (data_trabajo.Localidad == ''))
data_trabajo.loc[mask_Localidad, "calidad_dato"] = True

#resto_zonas
valores_zona_resto = ['Buenos Aires Costa Atlántica','Buenos Aires Interior','Capital Federal','Resto de provincias']
mask_Partido_barrio = ([x in valores_zona_resto for x in data_trabajo.Zona]) & ((data_trabajo.Partido_barrio.isnull())|\
                                                                                (data_trabajo.Partido_barrio == ''))
mask_calidad_dato_false = data_trabajo.calidad_dato == False
data_trabajo.loc[mask_Partido_barrio & mask_calidad_dato_false, "calidad_dato"] = True
# ----------------------------------------------------------------------
# todos los casos con True corresponden a los registros sobre los que nos falta informacion referida a 
# la geolocalización de la propiedad
print ('Cantidad de casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------')
print(data_trabajo.calidad_dato.value_counts(), '\n')
# mascara sobre casos que nos faltaria info
mask_faltante = (data_trabajo.calidad_dato == True)
data_faltante = data_trabajo.loc[mask_faltante]
#print (data_faltante)
# agrupo por Pais, Zona, Partido_barrio los casos que no tenemos otra info
groupby_df_faltante = data_faltante.groupby(['Pais',  'Zona', 'Partido_barrio'])["Id_caso"].count().\
sort_values(ascending=False)
print ('Agrupación de los casos a los que le falta información de localización:')
print ('-----------------------------------------------------------------------')
groupby_df_faltante


Cantidad de casos a los que le falta información de localización:
-----------------------------------------------------------------
False    107421
True      13799
Name: calidad_dato, dtype: int64 

Agrupación de los casos a los que le falta información de localización:
-----------------------------------------------------------------------


Pais       Zona                          Partido_barrio     
Argentina  Bs.As. G.B.A. Zona Norte      Tigre                  2359
                                         Pilar                  1857
           Capital Federal                                      1297
           Bs.As. G.B.A. Zona Norte      Escobar                1037
           Bs.As. G.B.A. Zona Sur        La Plata                767
           Bs.As. G.B.A. Zona Norte      San Isidro              641
           Bs.As. G.B.A. Zona Oeste      Morón                   582
                                         Ituzaingó               474
           Bs.As. G.B.A. Zona Norte      San Fernando            437
           Bs.As. G.B.A. Zona Oeste      Moreno                  434
           Bs.As. G.B.A. Zona Norte      San Miguel              417
                                         General San Martín      382
           Bs.As. G.B.A. Zona Sur        Lomas de Zamora         276
           Bs.As. G.B.A. Zona Oeste      M

***Serie*** `geonames_id`

Id asignado a cada `place_name`

Primero: vamos a validar que NO se asigne un mismo `geonames_id` a distintos valores de `place_name_new`

In [37]:
mask_geonames_id_nonull = data_trabajo.geonames_id.notnull()
df_geonames_id_place_name_ag = data_trabajo.loc[mask_geonames_id_nonull]

grouped = df_geonames_id_place_name_ag.groupby(['geonames_id',  'place_name_new'])
grouped1 = grouped['Id_caso'].agg([np.size])

df_agrupado2 = grouped1.groupby(['geonames_id']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado2)

Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
                                          size
geonames_id place_name_new                    
3428927.0   San Jose                         5
            San José                        11
3430234.0   Palermo                       2885
            Palermo Soho                   394
3433359.0   Ituzaingó                      832
            Villa Udaondo                   25
3435548.0   Centro                         231
            Centro / Microcentro           223
3435907.0   Bs.As. G.B.A. Zona Norte       222
            Bs.As. G.B.A. Zona Oeste        65
            Bs.As. G.B.A. Zona Sur          24
            Buenos Aires Costa Atlántica    27
            Buenos Aires Interior          106


Existen 5 valores de `geonames_id` que tiene mas de un valor de `place_name` por lo que nos vemos en la necesidad de analizar un poco mas, para ver si corresponden a la misma zona y ver que trabajo hacemos sobre ellos


In [38]:
print ('Comenzamos con el proceso de análisis y actualización de los los casos:')
print ('-----------------------------------------------------------------------')
print ('geonames_id = 3428927')
print ('----------------------')
# geonames_id = 3428927
    # mascara de id
mask_geonames_id  = data_trabajo.geonames_id == 3428927
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|','|Argentina|Bs.As. G.B.A. Zona Sur|\
Almirante Brown|San Jose|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"')
print ('Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|')
print ('Actualizamos la variable Partido_barrio = "Almirante Brown"')
print ('Actualizamos la variable Localidad = "San Jose"')
print ('Actualizamos la variable place_name_new = "San Jose"')
    
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # validamos en google que la localidad de 'San Jose' corresponde al partido de 'Almirante Brown'
    # para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
    # actualizamos la variable Partido_barrio = 'Almirante Brown'
    # actualizamos la variable Localidad = 'San Jose'
    # actualizamos la variable place_name_new = 'San Jose'

# creamos una veriable igual a geonames_id sobre le df para trabajar sobre esa
data_trabajo['geonames_id_new'] = data_trabajo.geonames_id
# mascara de geonames_id 
mask_geonames_id  = data_trabajo.geonames_id == 3428927
mask_place_with_parent_names = data_trabajo.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Sur|\
Lomas de Zamora|San José|'
# actualizamos el valor de Partido_barrio para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_antes.Partido_barrio.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']] = 'Almirante Brown'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Partido_barrio']]
print (data_despues.Partido_barrio.value_counts())
# actualizamos el valor de Localidad para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_antes.Localidad.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['Localidad']]
print (data_despues.Localidad.value_counts())
# actualizamos el valor de place_name_new para los casos de la mascara
print ('Cantidad de casos para actualizar:')
data_antes =  data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_antes.place_name_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']] = 'San Jose'
print ('Cantidad de casos actualizados:')
data_despues = data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names,['place_name_new']]
print (data_despues.place_name_new.value_counts())



Comenzamos con el proceso de análisis y actualización de los los casos:
-----------------------------------------------------------------------
geonames_id = 3428927
----------------------
|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|    11
|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Brown|San Jose|     5
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3428927.0    16
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Validamos en google que la localidad de "San Jose" corresponde al partido de "Almirante Brown"
Para esos registros mask_place_with_parent_names = |Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zamora|San José|
Actualizamos la variable Partido_barrio = "Almirante Brown"
Actualizamos la variable Localidad = "San Jose"
Actualizamos la variable place_name_new = "San Jose"
Cantidad de casos para actualizar:
Lom

In [14]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3430234')
print ('----------------------')
mask_geonames_id  = data_trabajo.geonames_id == 3430234
# traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names']]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Capital Federal|Palermo|','|Argentina|Capital Federal|Palermo|Palermo Soho|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo') 
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id_new en nulo para el caso de 'Palermo Soho' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Capital Federal|Palermo|Palermo Soho|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3430234
----------------------
|Argentina|Capital Federal|Palermo|                 2885
|Argentina|Capital Federal|Palermo|Palermo Soho|     394
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3430234.0    3279
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id_new en nulo para el caso de "Palermo Soho" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3430234.0    394
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [15]:
#..................................
# geonames_id =  3433359
    # mascara de id
mask_geonames_id  = data_trabajo.geonames_id == 3433359
print ('geonames_id = 3433359')
print ('---------------------')
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|','|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id']]
print (ver1.geonames_id.value_counts())
print ('Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Villa Udaondo' que es mas abarcativa para sumarlo 
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())


geonames_id = 3433359
---------------------
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|                  474
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Ituzaingó|        358
|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|     25
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3433359.0    857
Name: geonames_id, dtype: int64
Resultado ... como son de la misma zona y no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Villa Udaondo" que es mas abarcativa para sumarlo
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3433359.0    25
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [16]:
#..................................
# geonames_id = 3430234, 3433359, 3435548
    # mascara de id
print ('geonames_id = 3435548')
print ('---------------------')
mask_geonames_id  = data_trabajo.geonames_id == 3435548
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|','|Argentina|Capital Federal|Centro / Microcentro|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para el caso de 'Mar del Plata' solo para por simple elección y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
mask_place_with_parent_names1 = data_trabajo.place_with_parent_names == '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|'
data_antes =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id & mask_place_with_parent_names1, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())



geonames_id = 3435548
---------------------
|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|    231
|Argentina|Capital Federal|Centro / Microcentro|                 223
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435548.0    454
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para el caso de "Mar del Plata" solo para por simple elección y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435548.0    231
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [17]:
#..................................
# geonames_id = 3435907
    # mascara de id
print ('geonames_id = 3435907')
print ('---------------------')
valores_geonames_id = [3435907]
mask_geonames_id  = ([x in valores_geonames_id for x in data_trabajo.geonames_id]) 
    # traemos la informacion de esos casos 
ver = data_trabajo.loc[mask_geonames_id,['place_with_parent_names'] ]
print (ver.place_with_parent_names.value_counts())
print ('Validamos que para esos place_with_parent_names no existan otros geonames_id')
    # Validamos que para esos place_with_parent_names no existan otros geonames_id
valores = ['|Argentina|Bs.As. G.B.A. Zona Norte|','|Argentina|Bs.As. G.B.A. Zona Oeste|','|Argentina|Bs.As. G.B.A. Zona Sur|','|Argentina|Buenos Aires Costa Atlántica|','|Argentina|Buenos Aires Interior|']
mask_place_with_parent_names  =  ([x in valores for x in data_trabajo.place_with_parent_names]) 
ver1 = data_trabajo.loc[mask_place_with_parent_names,['geonames_id'] ]
print (ver1.geonames_id.value_counts())
print ('Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id')
print ('Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar')
print ('Al tratamiente que vamos hacer despues con los nulos de geonames_id')
    # resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
    # vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
    # al tratamiente que vamos hacer despues con los nulos de geonames_id
# mascara
data_antes =   data_trabajo.loc[mask_geonames_id, ['geonames_id_new']]
print ('Cantidad original de casos para actualizar:')
print(data_antes.geonames_id_new.value_counts())
data_trabajo.loc[mask_geonames_id, ['geonames_id_new']] = None
print ('Cantidad posterior de casos para actualizar:')
data_despues =   data_trabajo.loc[mask_geonames_id, ['geonames_id_new']]
print(data_despues.geonames_id_new.value_counts())

geonames_id = 3435907
---------------------
|Argentina|Bs.As. G.B.A. Zona Norte|        222
|Argentina|Buenos Aires Interior|           106
|Argentina|Bs.As. G.B.A. Zona Oeste|         65
|Argentina|Buenos Aires Costa Atlántica|     27
|Argentina|Bs.As. G.B.A. Zona Sur|           24
Name: place_with_parent_names, dtype: int64
Validamos que para esos place_with_parent_names no existan otros geonames_id
3435907.0    444
Name: geonames_id, dtype: int64
Resultado ... en este caso no corresponde a la mimsa zona, pero no existen mask_place_with_parent_names para otros geonames_id
Vamos a dejar geonames_id en nulo para todos los casos y lo vamos a sumar
Al tratamiente que vamos hacer despues con los nulos de geonames_id
Cantidad original de casos para actualizar:
3435907.0    444
Name: geonames_id_new, dtype: int64
Cantidad posterior de casos para actualizar:
Series([], Name: geonames_id_new, dtype: int64)


In [18]:
import random
print('Variable geonames_id')
print('Vamos a analizar como podemos actualizar los valores nulos')
print('----------------------------------------------------------')
print('Cantidad de valores nulos: ' + str(data_trabajo.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos: ' + str(len(data_trabajo.geonames_id_new.unique())))
print('Valor mínimo: ' + str(data_trabajo.geonames_id_new.min()))
print('Valor máximo: ' + str(data_trabajo.geonames_id_new.max()))
# vamos a trabajar los nulos del campo geonames_id_new\n# y lo hacemos por zonas:
# definimos 3 zonas de acuerdo al analisis del dato faltante de los procesos sobre place_with_parent_names
# valores_geo_grupo1  = \'Bs.As. G.B.A. Zona Norte\',\'Bs.As. G.B.A. Zona Oeste\',\'Bs.As. G.B.A. Zona Sur\',\'Capital Federal\'\n# mas valores_geo_interior\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio asigamos un numero al azar que identificara el place_name\n
# valores_geo_grupo2 = \'Buenos Aires Costa Atlántica\', \'Buenos Aires Interior\'\n    
    # si no tiene sino tiene Partido_barrio informado lo dejamos nulo ya que no seria correcto asignar un valor ya que \n    
    # abarca demasiado territorio\n    
    # si tiene Partido_barrio y Localidad, y NO son iguales asigamos un numero al azar que identificara el place_name\n    
    # si tiene Partido_barrio y Localidad, y SI son iguales asigamos, buscamos el ID del Partido_barrio \n    
# que si está informado\n\n
# creamos los grupos\n
valores_geo_grupo1 = ['Bs.As. G.B.A. Zona Norte','Bs.As. G.B.A. Zona Oeste','Bs.As. G.B.A. Zona Sur','Capital Federal']
valores_geo_grupo2 = ['Buenos Aires Costa Atlántica', 'Buenos Aires Interior']
valores_geo_interior_temp = data_trabajo.Zona.unique()
valores_geo_interior_y_1 = list(set(valores_geo_interior_temp).difference(set(valores_geo_grupo2)))

mask_geo_grupo2  =  ([x in valores_geo_grupo2 for x in data_trabajo.Zona])
mask_geo_interior_y_1  =  ([x in valores_geo_interior_y_1 for x in data_trabajo.Zona])
mask_geo_null = data_trabajo.geonames_id_new.isnull()
#mask_Partido_barrio_notnull = data_trabajo.Partido_barrio.notnull()
mask_Partido_barrio_null =  (data_trabajo.Partido_barrio.isnull()) | (data_trabajo.Partido_barrio == '')
mask_Partido_barrio_notnull = mask_data_Partido_barrio_null == False
mask_PB_dist_Localidad = data_trabajo.Partido_barrio != data_trabajo.Localidad

# Creamos una serie con los distintos valores de place_name que no interesa aplicarle una numero al azar
groupby_place_name_null = data_trabajo.loc[(mask_geo_interior_y_1 & mask_geo_null & mask_Partido_barrio_notnull)\
                                   | (mask_geo_grupo2&mask_geo_null&mask_Partido_barrio_notnull&mask_PB_dist_Localidad)\
                                   ].groupby(['place_name_new'])["Id_caso"].count()
# convertimos la seria en un df para trabajarlo
groupby_place_name_null_df = groupby_place_name_null.to_frame()
# asignamos a cada valor el número aleatorio partiendo de un numero lo bastante grande para que
# coincida con los existentes
groupby_place_name_null_df["geonames_id_temp"] = groupby_place_name_null_df.apply(lambda x: random.randint(10000000, 20000000), axis=1)

# creamos un nuevo df sumando el campo nuevo 
datay = pd.merge(left=data_trabajo,right=groupby_place_name_null_df, how='left', left_on='place_name_new', \
                 right_on='place_name_new')
#-------------------------------------------------------------------------------------------------------------
# actualizamos los casos que convertimos a nulo en el arreglo de geonames_id
#mask_geo_temp = ['|Argentina|Capital Federal|Palermo|Palermo Soho|']
#mask_geo_temp1  =  ([x in mask_geo_temp for x in datay.place_with_parent_names])
#max_valor = datay.geonames_id_temp.max()
#max_valor = max_valor + 1
#print(max_valor)
#datay.loc[mask_geo_temp1, 'geonames_id_temp'] = datay.loc[mask_geo_temp1, 'geonames_id_temp'].apply(lambda x: max_valor)
#['|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|']
#['|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|Villa Udaondo|']
#-------------------------------------------------------------------------------------------------------------

mask_geo_null = data_trabajo.geonames_id_new.isnull()
datay.loc[mask_geo_null, 'geonames_id_new'] = datay.loc[mask_geo_null, 'geonames_id_temp']
del datay['Id_caso_y']
datay.rename(columns={'Id_caso_x': 'Id_caso'}, inplace=True)
print('Cantidad de valores nulos final: ' + str(datay.geonames_id_new.isnull().sum()))
print('Cantidad de valores distintos final: ' + str(len(datay.geonames_id_new.unique())))

#Validamos que luego de la aplicacion de nulos no hayan quedado para un mismo 
# geonames_id a distintos valores de place_name_new

# variable geonames_id,  corresponde al id asignado a cada place_name
# volvemos a validar que No se asigne un mismo geonames_id a distintos valores de place_name_new
mask_geonames_id_new_notnull = datay.geonames_id_new.notnull()
df_geonames_id_place_name_ag2 = datay.loc[mask_geonames_id_new_notnull]

grouped_0 = df_geonames_id_place_name_ag2.groupby(['geonames_id_new',  'place_name_new'])
grouped_1 = grouped_0['Id_caso'].agg([np.size])

df_agrupado_2 = grouped_1.groupby(['geonames_id_new']).filter(lambda grp: grp["size"].count() > 1)
print ('Casos que un mismo geonames_id tiene asignado más de un place_name')
print ('------------------------------------------------------------------')
print (df_agrupado_2)

data_trabajo = datay
data_trabajo.drop("geonames_id_temp", axis = 1, inplace=True)

print ('Casos con geonames_id nulo')
print ('--------------------------')
# actualizo la mascara de los nulos
mask_geo_null = data_trabajo.geonames_id_new.isnull()
print (data_trabajo.loc[mask_geo_null, 'place_with_parent_names'].value_counts())




Variable geonames_id
Vamos a analizar como podemos actualizar los valores nulos
----------------------------------------------------------
Cantidad de valores nulos: 19811
Cantidad de valores distintos: 646
Valor mínimo: 3427208.0
Valor máximo: 6948895.0
Cantidad de valores nulos final: 610
Cantidad de valores distintos final: 1107
Casos que un mismo geonames_id tiene asignado más de un place_name
------------------------------------------------------------------
Empty DataFrame
Columns: [size]
Index: []
Casos con geonames_id nulo
--------------------------
|Argentina|Bs.As. G.B.A. Zona Norte|                                     222
|Argentina|Buenos Aires Interior|                                        106
|Argentina|Tucumán|                                                       77
|Argentina|Bs.As. G.B.A. Zona Oeste|                                      65
|Argentina|Buenos Aires Interior|Chascomús|Chascomús|                     54
|Argentina|Buenos Aires Costa Atlántica|           

In [129]:
print ('Proceso de validación de los campos Lat-Lon vs Lat / Lon')
print ('--------------------------------------------------------')
# validamos que los campos Lat y Lon tengan la misma cantidad y valor de info que Lat-Lon
# primero separamos el campo Lat-Lon
df_lat_lon  = data_trabajo.loc[:,["Id_caso","lat-lon", "lat", "lon"]].join(data["lat-lon"].str.split(',', expand=True).rename(columns={
                                                                                                0:'lat_temp', 
                                                                                                1:'lon_temp'}))
# los campos saparados los convierto en numéricos
df_lat_lon.lat_temp = df_lat_lon.lat_temp.apply(pd.to_numeric)
df_lat_lon.lon_temp = df_lat_lon.lon_temp.apply(pd.to_numeric)

# Validmos que los campos tengan la misma cantidad de NO nulos
print (df_lat_lon[["lat-lon", "lat", "lon"]].notnull().sum())
# mascara de no nulos
mask_notnull = df_lat_lon["lat-lon"].notnull()
# mascara de valores iguales 
ver_lat = round(df_lat_lon.lat,6) == round(df_lat_lon.lat_temp,6)
ver_lon = round(df_lat_lon.lon,6) == round(df_lat_lon.lon_temp,6)
# validamos la cantidad de casos
print ('Cantidad de casos iguales latitud: ' + str(ver_lat.sum()))
print ('Cantidad de casos iguales longitud: ' + str(ver_lon.sum()))


Proceso de validación de los campos Lat-Lon vs Lat / Lon
--------------------------------------------------------
lat-lon    69670
lat        69670
lon        69670
dtype: int64
Cantidad de casos iguales latitud: 69502
Cantidad de casos iguales longitud: 69396


In [21]:
print ('Proceso de validación de tipo de cambio')
print ('---------------------------------------')
# proceso de validación del tipo de cambio para validar si es correcto tomar el 
# el campo price_aprox_usd para todos los casos

# validamos que si currency (moneda) = 
# nulo  ----> nulo
# 'USD'  ----> price_aprox_local_currency / price  ----> valor por dolar de precios en dolares
# sino ---->price_aprox_local_currency / price_aprox_usd ----> valor por dolar de precios en pesos
# agrupamos los valores por dolar ... redondeado a 2 dígitos

#.................................................
print ('Distintos tipos de moneda:')
# Vemos los distintos valores de la variable moneda
print(data_trabajo.currency.value_counts())
# vemos que existen 3 registros con valores PEN (sol peruano) y UYU (peso Uruguayo)
# vemos los casos
print ('Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos')
valores_otras_monedas = ['PEN', 'UYU']
mask_otras_monedas = [x in valores_otras_monedas for x in data_trabajo.currency]
data_trabajo.loc[mask_otras_monedas, ["currency","price"]] = None

# sumamos la varible al Df Valor_dolar
data_trabajo["Valor_dolar"] = 0
# trabajamos la moneda = 'USD'
mask_dolar = data_trabajo.currency == 'USD'
mask_price = (data_trabajo.price != 0) | (data_trabajo.price.notnull())
data_trabajo.loc[mask_dolar & mask_price, "Valor_dolar"] = round(data_trabajo.price_aprox_local_currency / data_trabajo.price,2)
# trabajamos la moneda = 'ARG'
mask_peso = data_trabajo.currency == 'ARS'
mask_price_aprox_usd = (data_trabajo.price_aprox_usd != 0) | (data_trabajo.price_aprox_usd.notnull())
data_trabajo.loc[mask_peso & mask_price_aprox_usd, "Valor_dolar"] = round(data_trabajo.price_aprox_local_currency / data_trabajo.price_aprox_usd,2)
print ('Distintos valores del tipo de cambio luego de actualizar la variable creada')
# vemos los valores distintos para valor de dolar
print(data_trabajo.Valor_dolar.value_counts())
print ('Vemos que existe un solo valor ... Exc!')
print ('Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar')




Proceso de validación de tipo de cambio
---------------------------------------
Distintos tipos de moneda:
USD    87587
ARS    13219
PEN        2
UYU        1
Name: currency, dtype: int64
Los valores "PEN" y "UYU" los actualizamos a nulos para no descartarlos
Distintos valores del tipo de cambio luego de actualizar la variable creada
17.64    100806
0.00      20414
Name: Valor_dolar, dtype: int64
Vemos que existe un solo valor ... Exc!
Llegamos a la conclusion que es posible usar la variable price_aprox_usd como precio de la propiedad estándar


In [22]:
print ('Proceso de análisis del precio por m2')
print ('-------------------------------------')
print('Variables:')
print('- price_usd_per_m2')
print('- price_per_m2')
    
print ('Al ver alguno ejemplos vamos a validar que:')
print ('price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)')
print ('y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)')
print ('por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable')
    

# lo validamos
print ('Validación de price_usd_per_m2')
print ('------------------------------')
# ----------------------------------price_usd_per_m2
# sumamos un nuevo campo 
data_trabajo["price_usd_per_m2_new"] = -1
# mascara de los NO nulos price_aprox_usd
mask_price_aprox_usd_notnull = data_trabajo.price_aprox_usd.notnull()
# mascara de los no nulos y mayores a cero de surface_total_in_m2
mask_surface_total_in_m2_notnull_0 = ((data_trabajo.surface_total_in_m2.notnull()) & (data_trabajo.surface_total_in_m2 > 0))
# actualización de la variable nueva price_usd_per_m2_new
data_trabajo.loc[mask_price_aprox_usd_notnull & mask_surface_total_in_m2_notnull_0, "price_usd_per_m2_new"] = round(data_trabajo.price_aprox_usd/data_trabajo.surface_total_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_aprox_usd = data_trabajo.price_usd_per_m2_new != round(data_trabajo.price_usd_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 52606 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_aprox_usd.value_counts())
print('Vemos que hay 52606 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print(data_trabajo.loc[mask_validacion_price_aprox_usd, "price_usd_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_usd_per_m2 corresponde a')
print('a price_aprox_usd / surface_total_in_m2')

# ----------------------------------price_per_m2
print ('Validación de price_per_m2')
print ('------------------------------')
# sumamos un nuevo campo 
data_trabajo["price_per_m2_new"] = -1
# mascara de los NO nulos price
mask_price_notnull = data_trabajo.price.notnull()
# mascara de los no nulos y mayores a cero de surface_covered_in_m2
mask_surface_covered_in_m2_notnull_0 = ((data_trabajo.surface_covered_in_m2.notnull()) & (data_trabajo.surface_covered_in_m2 > 0))
# actualización de la variable nueva price_per_m2_new
data_trabajo.loc[mask_price_notnull & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data_trabajo.price/data_trabajo.surface_covered_in_m2,2)
# validamos creando una mascara sobre los que NO coinciden variable original vs variable nueva
mask_validacion_price_per_m2 = data_trabajo.price_per_m2_new != round(data_trabajo.price_per_m2,2)
print('Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:')
# agrupamos por la mascara y vemos que existen 33562 casos que son verdaderos .. NO coinciden
print(mask_validacion_price_per_m2.value_counts())
print('Vemos que hay 33562 que no coinciden')
print('Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara')
# contamos los nulos de esos no coincidentes y vemos que no coinciden porque la variable nueva se 
# le asigne como valor default -1 y en la variable original estan con nulo
print (data_trabajo.loc[mask_validacion_price_per_m2, "price_per_m2"].isnull().value_counts())
print('Conclusión: la variable price_per_m2 corresponde a')
print('price / surface_covered_in_m2')
# ahora vamos a actualizar la veriable que creamos price_per_m2 pero tomando para el caso de moneda ARG la 
# variable en dolares price_aprox_usd para que todo quede en la misma moneda
# mascara moneda ARG
mask_peso = data_trabajo.currency == 'ARS'
data_trabajo.loc[mask_peso & mask_surface_covered_in_m2_notnull_0, "price_per_m2_new"] = round(data_trabajo.price_aprox_usd/data_trabajo.surface_covered_in_m2,2)
# pasamos a nulo los que quedaron con valor -1
data_trabajo['price_usd_per_m2_new'].replace(-1, np.nan, inplace=True)
data_trabajo['price_per_m2_new'].replace(-1, np.nan, inplace=True)


Proceso de análisis del precio por m2
-------------------------------------
Variables:
- price_usd_per_m2
- price_per_m2
Al ver alguno ejemplos vamos a validar que:
price_usd_per_m2 corresponde al precio en dólares sobre la superficie total en m2(price_aprox_usd / surface_total_in_m2)
y que price_per_m2 corresponde al precio original sobre la superficie cubierta(price / surface_covered_in_m2)
por lo que sobre esta última, ademas vamos a estandarizarla a dólares sobre una nueva variable
Validación de price_usd_per_m2
------------------------------
Luego de "imitar" la variable agrupamos por la marca si coinciden o no con la variable original:
False    68614
True     52606
dtype: int64
Vemos que hay 52606 que no coinciden
Validamos que todos los casos que correspondan a los valores nulos, mediante una máscara
True     52603
False        3
Name: price_usd_per_m2, dtype: int64
Conclusión: la variable price_usd_per_m2 corresponde a
a price_aprox_usd / surface_total_in_m2
Validación de price

In [137]:
########### esto no se ejecuta!!!
## de aca en adelante son pruebas


BUSCAR DUPLICADOS EN UN DF
artist_duplicated = data_artist.duplicated(subset=["Artist Display Name", "Artist Role"])
any(artist_duplicated) * me devuelve verdadero o falso
artist_duplicated.sum()  * me devuelve la cantidad de duplicados
BORRO LOS DUPLICADOS
data_artist_unique = data_artist.drop_duplicates(subset=["Artist Display Name", "Artist Role"], keep="first")
keep ... 
* first: devuelve False (no duplicado) el primer registro encontrado, y True (duplicados) todas las demás apariciones
* last: devuelve False (no duplicado) el último registro encontrado, y True (duplicados) todas las demás apariciones
* False: devuelve True (duplicado) todos los registros duplicados

In [152]:
ver = data_trabajo.duplicated(subset=["currency"])
data_artist_unique = data_artist.drop_duplicates(subset=["Artist Display Name", "Artist Role"], keep="first")

0         False
1          True
2          True
3          True
4          True
          ...  
121215     True
121216     True
121217     True
121218     True
121219     True
Length: 121220, dtype: bool

In [ ]:
df_na.to_csv('data_output/surveys_complete.csv', index=False)

---

<a id="section_re"></a> 
## Expresiones Regulares

 Segun la cantidad de registros completados sobre las Series `title` `description` del dataset decidimos realizar una analisis exploratorio sobre los patrones de texto que hacen referencia a los siguientes elementos:
 - Precios de inmuebles en dolares.
 - Metros cuadrados.
 - Ambientes.
 - Amenities: cochera, balcon, parrilla, pileta/piscina, baulera, lavadero, terraza y jardin.
 
La busqueda de patrones de texto se realizo mediante el uso de expresiones regulares. En el armado de los patrones de busqueda se tuvo en consideracion el contexto donde se encontraba la cadena de caracteres a ser extraida para minimizar el numero de falsos positivos.

Luego se aplico el uso de outliers para identificar casos extremos que pudieran indicar la existencia de valores falsos.

[volver a TOC](#section_toc)

<a id="section_re_pg"></a> 
### **Principios generales**

[volver a TOC](#section_toc)

### Patrones

**Precios inmueble en dolares**
 - `(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)?\d{0,3}(\,|\.)?d{0,3}(\,|\.)?\d{0,3})[^\w] ?(dolares)?(?!(.*m2|.*mc|.*metros))`
 - Busqueda de precios con valores entre 10000 y 99999999999.
 - Separacion de decimas con punto (.) o coma(,).
 - Debe indicarse prefijo en dolares
 - No debe contener como sufijo una denominacion a metros cuadrados.

**Metros inmueble**
 - `(\d{1,4}\,?\.?\d{0,4}) ?(m2|mc|metros|mts)`
 - VER SI AGREGAR.

**Ambientes**
 - `(?<!\-) ([1-2][0-9]?)(?= amb[^o])`
 - Entre 1 a 29 ambientes.
 - Debe tener como subfijo la cadena de caracteres "amb" o "ambientes" (no puede tener, por ejemplo "ambos") 
 - No debe ser precedido por guiones, ejemplo: 100-80 amb.

**Cochera**
 - `(?<!sin )(?<!no incluye )(?<!no posee )cochera(?! no incluye)(?! no incluida)`.
 - Busqueda de cadena de caracteres "cochera" o "cocheras".
 - La cadena de caracteres no puede estar precedida y sucedida por elementos que nieguen su existencia.

**Piscina/Pileta**
 - `piscina|pileta(?! de baño)(?! de cocina)`
 - Busqueda de cadena de caracteres "cochera" o "cocheras".
 - Se realiza la aclaracion de que la pileta no debe ser de baño o de cocina.

**Parilla**
 - `(?<!sin )(?<!no incluye )(?<!no posee )parrilla(?! no incluye)(?! no incluida)`
 - Busqueda de cadena de caracteres "parrilla" o "parrillas".
 - La cadena de caracteres no puede estar precedida y sucedida por elementos que nieguen su existencia.

**Baulera**
 - `(?<!sin )(?<!no incluye )(?<!no posee )Baulera(?! no incluye)(?! no incluida)`
 - Busqueda de cadena de caracteres "baulera" o "bauleras".
 - La cadena de caracteres no puede estar precedida y sucedida por elementos que nieguen su existencia.

Patrones

In [6]:
# Precios inmueble
precios_inmueble_pat = '(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)?\d{0,3}(\,|\.)?d{0,3}(\,|\.)?\d{0,3})[^\w] ?(dolares)?(?!(.*m2|.*mc|.*metros))'
precios_inmueble_reg = re.compile(precios_inmueble_pat, flags = re.IGNORECASE)

# Metros inmueble
metros_inmueble_pat = '(\d{1,4}\,?\.?\d{0,4}) ?(m2|mc|metros|mts)'
metros_inmueble_reg = re.compile(metros_inmueble_pat, flags = re.IGNORECASE)

# Ambientes - hay propiedaddes de 20 ambientes o mas?
#amb_pat = '(\d{1,2})(?= ambientes)'
amb_pat = '(?<!\-) ([1-2][0-9]?)(?= amb[^o])' # version mejorada
amb_reg = re.compile(amb_pat, flags = re.IGNORECASE)

# Cochera
cochera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )cochera(?! no incluye)(?! no incluida)'
cochera_reg = re.compile(cochera_pat, flags = re.IGNORECASE)

# Piscina/Pileta
piscina_pat = 'piscina|pileta(?! de baño)(?! de cocina)'
piscina_reg = re.compile(piscina_pat, flags = re.IGNORECASE)

# Parilla
parrilla_pat = '(?<!sin )(?<!no incluye )(?<!no posee )parrilla(?! no incluye)(?! no incluida)'
parrilla_reg = re.compile(parrilla_pat, flags = re.IGNORECASE)

# Baulera
baulera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )Baulera(?! no incluye)(?! no incluida)'
baulera_reg = re.compile(baulera_pat, flags = re.IGNORECASE)

## Regex Precios
---

In [47]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    100810
True      20410
Name: price_aprox_usd, dtype: int64

Campo Titulo

In [48]:
df_propierati['precios_normal_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else precios_inmueble_reg.search(x))
df_propierati['precios_normal_regex_titulo'] = df_propierati.precios_normal_regex_titulo \
    .apply(lambda x: 0 if x is None else x.group(2) \
           .replace('.','') \
           .replace(',','')) \
    .astype(int)

In [49]:
df_propierati['precios_normal_regex_titulo'].notnull().value_counts()

True    121220
Name: precios_normal_regex_titulo, dtype: int64

In [50]:
# analisis de outliers para identificar que eliminar
precios_normal_regex_titulo = df_propierati.precios_normal_regex_titulo.apply(lambda x: x if x > 30000 and x < 35000000 else np.NaN)
df_propierati['precios_normal_regex_titulo'] = df_propierati.precios_normal_regex_titulo.apply(lambda x: x if x > 30000 and x < 35000000 else np.NaN)

In [51]:
df_propierati['precios_normal_regex_titulo'].notnull().value_counts()

False    118601
True       2619
Name: precios_normal_regex_titulo, dtype: int64

Campo Descripcion

In [52]:
df_propierati['precios_normal_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else precios_inmueble_reg.search(x))
df_propierati['precios_normal_regex_desc'] = df_propierati.precios_normal_regex_desc \
    .apply(lambda x: 0 if x is None or x is np.NaN else x.group(2) \
           .replace('.','') \
           .replace(',','')) \
    .astype(int)

In [53]:
df_propierati['precios_normal_regex_desc'].notnull().value_counts()

True    121220
Name: precios_normal_regex_desc, dtype: int64

In [54]:
precios_normal_regex_desc = df_propierati.precios_normal_regex_desc.apply(lambda x: x if x > 30000 and x < 20000000 else np.NaN)
df_propierati['precios_normal_regex_desc'] = df_propierati.precios_normal_regex_desc.apply(lambda x: x if x > 30000 and x < 20000000 else np.NaN)

In [55]:
df_propierati['precios_normal_regex_desc'].notnull().value_counts()

False    115327
True       5893
Name: precios_normal_regex_desc, dtype: int64

In [56]:
print(df_propierati['precios_normal_regex_titulo'].apply(lambda x: type(x)).value_counts(), '\n', df_propierati['precios_normal_regex_titulo'].dtypes)

<class 'float'>    121220
Name: precios_normal_regex_titulo, dtype: int64 
 float64


In [57]:
print(df_propierati['precios_normal_regex_desc'].apply(lambda x: type(x)).value_counts(), '\n', df_propierati['precios_normal_regex_desc'].dtypes)

<class 'float'>    121220
Name: precios_normal_regex_desc, dtype: int64 
 float64


In [58]:
df_propierati['precios_normal_regex_desc'].size, df_propierati['precios_normal_regex_titulo'].size

(121220, 121220)

In [59]:
price_aprox_usd_null_mask = df_propierati.price_aprox_usd.isnull()

In [60]:
df_propierati.loc[price_aprox_usd_null_mask, 'price_aprox_usd'] = df_propierati.loc[price_aprox_usd_null_mask, 'precios_normal_regex_titulo']

In [61]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    103188
True      18032
Name: price_aprox_usd, dtype: int64

In [62]:
price_aprox_usd_null_mask = df_propierati.price_aprox_usd.isnull()

In [63]:
df_propierati.loc[price_aprox_usd_null_mask, 'price_aprox_usd'] = df_propierati.loc[price_aprox_usd_null_mask, 'precios_normal_regex_desc']

In [64]:
df_propierati.price_aprox_usd.isnull().value_counts()

False    103310
True      17910
Name: price_aprox_usd, dtype: int64

## Regex ambientes
---

In [28]:
df_propierati.rooms.apply(lambda x: type(x)).value_counts()

<class 'float'>    121220
Name: rooms, dtype: int64

In [29]:
df_propierati_rooms_null_mask = df_propierati.rooms.isnull()
df_propierati.rooms.isnull().value_counts()

True     73830
False    47390
Name: rooms, dtype: int64

In [30]:
df_propierati['rooms_regex_titl'] = df_propierati.title.apply(lambda x: x if x is np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_titl'] = df_propierati.rooms_regex_titl.apply(lambda x: x if x is None else x.group(1))
df_propierati['rooms_regex_titl'] = df_propierati.rooms_regex_titl.apply(lambda x: np.NaN if x is None else float(x))

In [31]:
df_propierati.rooms_regex_titl.isnull().value_counts()

True     114860
False      6360
Name: rooms_regex_titl, dtype: int64

In [32]:
df_propierati.rooms_regex_titl.value_counts()

2.0     5333
1.0     1019
11.0       3
10.0       2
15.0       1
14.0       1
12.0       1
Name: rooms_regex_titl, dtype: int64

In [33]:
df_propierati['rooms_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else amb_reg.search(x))
df_propierati['rooms_regex_desc'] = df_propierati.rooms_regex_desc.apply(lambda x: x if x is np.NaN or x is None else x.group(1))
df_propierati['rooms_regex_desc'] = df_propierati.rooms_regex_desc.apply(lambda x: np.NaN if x is None else float(x))

In [34]:
df_propierati.rooms_regex_desc.isnull().value_counts()

True     106492
False     14728
Name: rooms_regex_desc, dtype: int64

In [35]:
df_propierati.rooms_regex_desc.value_counts()

2.0     12482
1.0      2210
10.0       13
11.0       12
18.0        3
14.0        3
12.0        2
17.0        1
25.0        1
22.0        1
Name: rooms_regex_desc, dtype: int64

In [36]:
df_propierati.loc[df_propierati_rooms_null_mask, 'rooms'] = df_propierati.loc[df_propierati_rooms_null_mask, 'rooms_regex_titl']
df_propierati.rooms.isnull().value_counts()

True     71455
False    49765
Name: rooms, dtype: int64

In [37]:
df_propierati_rooms_null_mask = df_propierati.rooms.isnull()

In [38]:
df_propierati.loc[df_propierati_rooms_null_mask, 'rooms'] = df_propierati.loc[df_propierati_rooms_null_mask, 'rooms_regex_desc']
df_propierati.rooms.isnull().value_counts()

True     65802
False    55418
Name: rooms, dtype: int64

## Regex Metros
---

In [7]:
df_propierati.surface_total_in_m2.apply(lambda x: type(x)).value_counts()

<class 'float'>    121220
Name: surface_total_in_m2, dtype: int64

In [8]:
df_propierati.surface_total_in_m2.isnull().value_counts()

False    81892
True     39328
Name: surface_total_in_m2, dtype: int64

In [9]:
# analisis de outliers para identificar que eliminar

df_propierati['totalm2_normal_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else metros_inmueble_reg.search(x))
df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: x if x is None else x.group(1))

#df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: x if x is None else x.group(1).replace(',','.'))

#df_propierati['totalm2_normal_regex_titulo'] = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: 0 if x is np.NaN else x)
#f_propierati['totalm2_normal_regex_titulo'] = df_propierati['totalm2_normal_regex_titulo'].astype(int)

#df_propierati.loc[55106, 'totalm2_normal_regex_titulo']
#df_propierati.loc[17768, ['totalm2_normal_regex_titulo', 'title']]


df_propierati.surface_total_in_m2.value_counts()
#pepe = df_propierati.totalm2_normal_regex_titulo.apply(lambda x: np.NaN if x is None else float(x))
#df_propierati.totalm2_normal_regex_titulo.value_counts()
#pepe.sort_values(ascending=True).head(100)
#df_propierati['totalm2_normal_regex_titulo'].apply(lambda x: type(x)).value_counts()
#df_propierati['totalm2_normal_regex_titulo'].isnull().value_counts()
df_propierati['totalm2_normal_regex_titulo'].sort_values(ascending=True).head(200)
#df_propierati['totalm2_normal_regex_titulo'].value_counts()

27520          1
56822          1
56828          1
32984      1.000
14561      1.000
68526      1.000
68497      1.000
68477      1.000
11124      1.013
68583      1.100
66342      1.100
69007      1.133
65183      1.144
64804      1.200
66425      1.200
65122      1.230
64808      1.255
65791      1.270
97448      1.300
66298      1.340
64901      1.400
65270      1.500
97611      1.500
96283      1.500
66012      1.622
68727      1.707
115251     1.720
65845      1.750
65532      1.800
67016      1.890
97524      1.892
65095      1.950
73961         10
81026         10
22474         10
7592          10
33510         10
42575         10
82953         10
30576         10
69115     10.960
42665        100
7873         100
80921        100
51234        100
96776        100
24622        100
51527        100
80777        100
52324        100
32090        100
80538        100
52338        100
51147        100
6501         100
78009        100
42532        100
52820        100
53344        1

In [78]:
s = pd.Series(['98.5', '49.00', '85.74', np.NaN])
s = s.astype(float)
s

0    98.50
1    49.00
2    85.74
3      NaN
dtype: float64

---

## Regex amenities
---

### Cocheras

Titulo

In [11]:
df_propierati['cochera_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else cochera_reg.search(x))
df_propierati['cochera_regex_titulo'] = df_propierati.cochera_regex_titulo.apply(lambda x: x if x is None else x.group())

In [13]:
cochera_regex_titulo_notnull_mask = df_propierati['cochera_regex_titulo'].notnull()
df_propierati.loc[cochera_regex_titulo_notnull_mask, 'cochera_regex_titulo'] = 1
cochera_regex_titulo_null_mask = df_propierati['cochera_regex_titulo'].isnull()
df_propierati.loc[cochera_regex_titulo_null_mask, 'cochera_regex_titulo'] = np.NaN

df_propierati['cochera_regex_titulo'].isnull().value_counts()

True     115235
False      5985
Name: cochera_regex_titulo, dtype: int64

Descripcion

In [14]:
df_propierati['cochera_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else cochera_reg.search(x))
df_propierati['cochera_regex_desc'] = df_propierati.cochera_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [15]:
cochera_regex_desc_notnull_mask = df_propierati['cochera_regex_desc'].notnull()
df_propierati.loc[cochera_regex_desc_notnull_mask, 'cochera_regex_desc'] = 1
cochera_regex_desc_null_mask = df_propierati['cochera_regex_desc'].isnull()
df_propierati.loc[cochera_regex_desc_null_mask, 'cochera_regex_desc'] = np.NaN

df_propierati['cochera_regex_desc'].isnull().value_counts()

True     79062
False    42158
Name: cochera_regex_desc, dtype: int64

In [17]:
df_propierati['cochera'] = df_propierati['cochera_regex_titulo']
df_propierati.loc[df_propierati['cochera_regex_titulo'].isnull(), 'cochera'] = df_propierati.loc[df_propierati['cochera_regex_titulo'].isnull(), 'cochera_regex_desc']
df_propierati['cochera'].isnull().value_counts()

True     78075
False    43145
Name: cochera, dtype: int64

### Piscinas

Titulo

In [18]:
df_propierati['piscina_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else piscina_reg.search(x))
df_propierati['piscina_regex_titulo'] = df_propierati.piscina_regex_titulo.apply(lambda x: x if x is None else x.group())

In [19]:
piscina_regex_titulo_notnull_mask = df_propierati['piscina_regex_titulo'].notnull()
df_propierati.loc[piscina_regex_titulo_notnull_mask, 'piscina_regex_titulo'] = 1
piscina_regex_titulo_null_mask = df_propierati['piscina_regex_titulo'].isnull()
df_propierati.loc[piscina_regex_titulo_null_mask, 'piscina_regex_titulo'] = np.NaN

df_propierati['piscina_regex_titulo'].isnull().value_counts()

True     119616
False      1604
Name: piscina_regex_titulo, dtype: int64

Descripcion

In [20]:
df_propierati['piscina_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else piscina_reg.search(x))
df_propierati['piscina_regex_desc'] = df_propierati.piscina_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [21]:
piscina_regex_desc_notnull_mask = df_propierati['piscina_regex_desc'].notnull()
df_propierati.loc[piscina_regex_desc_notnull_mask, 'piscina_regex_desc'] = 1
piscina_regex_desc_null_mask = df_propierati['piscina_regex_desc'].isnull()
df_propierati.loc[piscina_regex_desc_null_mask, 'piscina_regex_desc'] = np.NaN

df_propierati['piscina_regex_desc'].isnull().value_counts()

True     88487
False    32733
Name: piscina_regex_desc, dtype: int64

In [22]:
df_propierati['piscina'] = df_propierati['piscina_regex_titulo']
df_propierati.loc[df_propierati['piscina_regex_titulo'].isnull(), 'piscina'] = df_propierati.loc[df_propierati['piscina_regex_titulo'].isnull(), 'piscina_regex_desc']
df_propierati['cochera'].isnull().value_counts()

True     78075
False    43145
Name: cochera, dtype: int64

### Parilla

Titulo

In [23]:
df_propierati['parrilla_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else parrilla_reg.search(x))
df_propierati['parrilla_regex_titulo'] = df_propierati.parrilla_regex_titulo.apply(lambda x: x if x is None else x.group())

In [24]:
parrilla_regex_titulo_notnull_mask = df_propierati['parrilla_regex_titulo'].notnull()
df_propierati.loc[parrilla_regex_titulo_notnull_mask, 'parrilla_regex_titulo'] = 1
parrilla_regex_titulo_null_mask = df_propierati['parrilla_regex_titulo'].isnull()
df_propierati.loc[parrilla_regex_titulo_null_mask, 'parrilla_regex_titulo'] = np.NaN

df_propierati['parrilla_regex_titulo'].isnull().value_counts()

True     120548
False       672
Name: parrilla_regex_titulo, dtype: int64

Descripcion

In [25]:
df_propierati['parrilla_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else parrilla_reg.search(x))
df_propierati['parrilla_regex_desc'] = df_propierati.parrilla_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [26]:
parrilla_regex_desc_notnull_mask = df_propierati['parrilla_regex_desc'].notnull()
df_propierati.loc[parrilla_regex_desc_notnull_mask, 'parrilla_regex_desc'] = 1
parrilla_regex_desc_null_mask = df_propierati['parrilla_regex_desc'].isnull()
df_propierati.loc[parrilla_regex_desc_null_mask, 'parrilla_regex_desc'] = np.NaN

df_propierati['parrilla_regex_desc'].isnull().value_counts()

True     87187
False    34033
Name: parrilla_regex_desc, dtype: int64

In [27]:
df_propierati['parrilla'] = df_propierati['parrilla_regex_titulo']
df_propierati.loc[df_propierati['parrilla_regex_titulo'].isnull(), 'parrilla'] = df_propierati.loc[df_propierati['parrilla_regex_titulo'].isnull(), 'parrilla_regex_desc']
df_propierati['parrilla'].isnull().value_counts()

True     87129
False    34091
Name: parrilla, dtype: int64

### Baulera

Titulo

In [35]:
df_propierati['baulera_regex_titulo'] = df_propierati.title.apply(lambda x: x if x is np.NaN else baulera_reg.search(x))
df_propierati['baulera_regex_titulo'] = df_propierati.baulera_regex_titulo.apply(lambda x: x if x is None else x.group())

In [36]:
baulera_regex_titulo_notnull_mask = df_propierati['baulera_regex_titulo'].notnull()
df_propierati.loc[baulera_regex_titulo_notnull_mask, 'baulera_regex_titulo'] = 1
baulera_regex_titulo_null_mask = df_propierati['baulera_regex_titulo'].isnull()
df_propierati.loc[baulera_regex_titulo_null_mask, 'baulera_regex_titulo'] = np.NaN

df_propierati['baulera_regex_titulo'].isnull().value_counts()

True     120693
False       527
Name: baulera_regex_titulo, dtype: int64

Descripcion

In [37]:
df_propierati['baulera_regex_desc'] = df_propierati.description.apply(lambda x: x if x is np.NaN else baulera_reg.search(x))
df_propierati['baulera_regex_desc'] = df_propierati.baulera_regex_desc.apply(lambda x: x if x is None or x is np.NaN else x.group())

In [38]:
baulera_regex_desc_notnull_mask = df_propierati['baulera_regex_desc'].notnull()
df_propierati.loc[baulera_regex_desc_notnull_mask, 'baulera_regex_desc'] = 1
baulera_regex_desc_null_mask = df_propierati['baulera_regex_desc'].isnull()
df_propierati.loc[baulera_regex_desc_null_mask, 'baulera_regex_desc'] = np.NaN

df_propierati['baulera_regex_desc'].isnull().value_counts()

True     110435
False     10785
Name: baulera_regex_desc, dtype: int64

In [39]:
df_propierati['baulera'] = df_propierati['baulera_regex_titulo']
df_propierati.loc[df_propierati['baulera_regex_titulo'].isnull(), 'baulera'] = df_propierati.loc[df_propierati['baulera_regex_titulo'].isnull(), 'baulera_regex_desc']
df_propierati['baulera'].isnull().value_counts()

True     110378
False     10842
Name: baulera, dtype: int64

---

In [40]:
df_propierati.head(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,cochera,piscina_regex_titulo,piscina_regex_desc,piscina,parrilla_regex_titulo,parrilla_regex_desc,parrilla,baulera_regex_titulo,baulera_regex_desc,baulera
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente López|Munro|,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN,NaN
9,9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
